# XIFUSIM Simulation of a real source (model spectra)   

It simulates a real source in all the pixels of X-IFU with `xifusim.`   
The parameters that define the simulation are (* for mandatory):   
    - Source flux in mCrab*   
    - Source model*   
    - filter* applied to defocused case:  `thinOpt` // `thickOpt` // `nofilt` // `thinBe` // `thickBe`   
    - focus: `infoc` or `defoc`. If not provided `defoc` is applied to flux > 0.5 flux_mcrab and `infoc` otherwise       
    - Lower energy in flux band (`Emin`): 2. keV   
    - Upper energy in flux band (`Emax`): 10. keV   

Other parameters (calculated/derived) are:   
    - RA of source: 0.   
    - Dec of source: 0.   
    - Exposure time: x times the interval required to have 2 close pulses (assuming Poissonian)


Possible models are:   
1. `CRAB`    
    Power law spectrum: $\Gamma=2.05$      
    Unabsorbed Flux(2-10keV): $21.6 \times 10^{-12} \rm{erg\,cm^{-2}\,s^{-1}}$     
    Foregroung absorption: $N_H=2\times 10^{21} \rm{cm^{-2}}$    
    XSPEC model: phabs*pegpwrlw   
2. `EXTEND` 

Simulation steps   
1. Read simulation parameters and derived parameters   
2. HEASOFT `xspec`: create xspec model file    
3. SIXTE `simputfile`: Create simput file with photons distribution    
4. SIXTE `sixtesim`: Run simulation to get   
    4.1 ImpactList - piximpact file for ALL photons 
    4.2 EventList - which photons (PH_ID) impact in each pixel of the detector (including background)   
    4.3 PixImpactList: piximpact file for each pixel with impacts (PH_ID) (needed by xifusim)   
5. Get list of pixels w/ impacts. For each pixel with >1 impact:   
    5.1. Check if there are "close" photons (otherwise skip xifusim simulation)   
    5.2. Create a sub-piximpact file from the total piximpact file with the close photons
    5.3. `xifusim`: Do single-pixel (NOMUX) xifusim simulation     
    5.3. `sirena`: reconstruct xifusim simulation   
    5.4. Analyse reconstruction to check for missing photons   
   

## Import routines and read parameters

In [ ]:
import ipywidgets as widgets 
%matplotlib widget

import os
from subprocess import run, PIPE, STDOUT
import sys
import tempfile
from datetime import datetime
from pathlib import Path
import glob

from auxiliary import vprint, time_to_observe_n_counts
from astropy.io import fits, ascii
from astropy.table import Table
import heasoftpy as hsp
import numpy as np
import  pandas as pd
from xspec import Xset, Plot, AllData, ModelManager, Spectrum, Model, AllModels, Fit


In [ ]:
tmpDir = tempfile.mkdtemp()
os.environ["PFILES"] = f"{tmpDir}:{os.environ['PFILES']}"
os.environ["HEADASNOQUERY"] = ""
os.environ["HEADASPROMPT"] = "/dev/null/"
SIXTE = os.environ["SIXTE"]
xmldir = f"{SIXTE}/share/sixte/instruments/athena-xifu/baseline"
xml = f"{xmldir}/xifu_nofilt_infoc.xml"

### Read command-line parameters   
```
sim_number: simulation run number   
flux_mcrab: erg/cm^2/s (1 mcrab=2.4E-11 erg/cm^2/s)   
Emin: (keV) to define the energy range of the flux   
Emax: (keV) to define the energy range of the flux   
model: "crab"//TBD   
filter:  thinOpt // thickOpt // nofilt // thinBe // thickBe    
focus: '' (TBD from flux)   
recons: 0 for no_reconstruction, 1 for do_reconstruction   
verbose: 0 (silent) or 1 (chatty)
```

In [ ]:
sim_number = 1
flux_mcrab = 0.5
Emin = 2.
Emax = 10.
model = "crab"
filter = "nofilt"
focus = ''
recons = 1
verbose = 1

### Read derived/extra parameters

In [ ]:
RA=0.
Dec=0.
sampling_rate=130210 #Hz
prebuff_xifusim=1500  #prebuffer samples for xifusim
pileup_dist=30 #samples for pileup
times_obstime=1000 # factor for time simulation
close_dist_toxifusim = 100 #samples for close events to decide if xifusim simulation will be done

In [ ]:
#calculate exposure to get counts 
# 1 mCrab = 90 counts/s in the 2-10 keV band
flux = flux_mcrab * 2.4E-11
rate = flux_mcrab * 90 #counts/s
time_30samps = pileup_dist/sampling_rate #s
npile = 2 #counts 
vprint(f"rate={rate} ct/s, time_interval(30 samples)={time_30samps:.3e}s, n={npile}photons")
observation_time = time_to_observe_n_counts(rate=rate, time_interval=time_30samps, n=npile)
vprint(f"Expected time to observe {npile} counts in 30 samples: {observation_time:.2f} seconds")
exposure = int(times_obstime * observation_time)
vprint(f"Using exposure time: {exposure:.2f}s")


### Create folder structure for output

In [ ]:
# create a new folder (if it does not exist) for the output using the flux as the name
if flux_mcrab < 0.01:
    flux_mcrab_str = f"{flux_mcrab:.2e}"
else:
    flux_mcrab_str = f"{flux_mcrab:.2f}"
fluxDir = f"flux{flux_mcrab_str}mcrab"
outDir = f"{fluxDir}/sim_{sim_number}"
outDirPath = f"{os.getcwd()}/{outDir}"
if not os.path.exists(outDirPath):
    os.makedirs(outDirPath)

### Set `sixtesim` XML file based on parameters (focus and filter)

In [ ]:
## Set XML file based on parameters
# if 'focus' is not provided: get it automatically according to the FLUX
# Filter will only be applied to the defocussed case
if focus == '':
    if flux_mcrab <= 0.5:
        focus="infoc"
        xml_sixtesim = f"{xmldir}/xifu_nofilt_{focus}.xml"
    else:
        focus="defoc"
        xml_sixtesim = f"{xmldir}/xifu_{filter}_{focus}.xml"
elif focus == "defoc":
    xml_sixtesim = f"{xmldir}/xifu_{filter}_defoc.xml"
elif focus == "infoc":
    xml_sixtesim = f"{xmldir}/xifu_nofilt_infoc.xml"

vprint(f"Using XML file: {xml_sixtesim}")

In [ ]:
# set string to name files based on input parameters
filestring_simput = f"./{fluxDir}/{model}_flux{flux_mcrab_str}_Emin{Emin:.0f}_Emax{Emax:.0f}_RA{RA}_Dec{Dec}"
filestring = f"./{outDir}/{model}_flux{flux_mcrab_str}_Emin{Emin:.0f}_Emax{Emax:.0f}_exp{exposure}_RA{RA}_Dec{Dec}_{filter}_{focus}"
#filestring = f"{filestring_simput}_{filter}_{focus}"
print(filestring)

## Create XSPEC model file   

In [ ]:
# is spectral model file does not exist, create it
if model == "crab":
    xcm = f"{model}.xcm"
    if not os.path.exists(xcm):
        # Clear all models
        AllModels.clear()
        # define XSPEC parameters
        Xset.abund = "wilm"
        Xset.cosmo = "70 0. 0.73"
        Xset.xsect = "bcmc"
        mcmod = Model("phabs*pegpwrlw")
        mcmod.phabs.nH = 0.2
        mcmod.pegpwrlw.PhoIndex = 2.05
        mcmod.pegpwrlw.eMin = 2.
        mcmod.pegpwrlw.eMax = 10.
        mcmod.pegpwrlw.norm = 1.
        #retrieve the flux value
        AllModels.calcFlux(f"{Emin} {Emax}")
        model_flux = AllModels(1).flux[0]
        # calculate the new norm value
        new_norm = flux/model_flux
        mcmod.pegpwrlw.norm = new_norm
        # Save the model to the specified .xcm file path
        Xset.save(xcm)
        vprint(f"Model saved to {xcm}")
else:
    vprint("Model not implemented yet")
    sys.exit(1)
#mcmod.show()

## Create simput file

In [ ]:
# run simputfile to create the simput file
simputfile = f"{filestring_simput}_simput.fits"
if not os.path.exists(simputfile):
        comm = (f'simputfile Simput={simputfile} RA={RA} Dec={Dec} '
                f'srcFlux={flux} Emin={Emin} Emax={Emax} '
                f'XSPECFile={xcm} clobber=yes')
        vprint(f"Running {comm}")
        # Run the command through the subprocess module
        output_simputfile = run(comm, shell=True, capture_output=True)
        #print(output_simputfile.stdout.decode())
        assert output_simputfile.returncode == 0, f"simputfile failed to run: {comm}"
        assert os.path.exists(simputfile), f"simputfile did not produce an output file"

## Run sixtesim simulation: Create PIXIMPACT file 

In [ ]:
evtfile = f"{filestring}_evt.fits"
photfile = f"{filestring}_photon.fits"
impfile = f"{filestring}_impact.fits"
if not os.path.exists(evtfile) or not os.path.exists(photfile) or not os.path.exists(impfile):    
        comm = (f'sixtesim PhotonList={photfile} Simput={simputfile} '
                f'ImpactList={impfile} EvtFile={evtfile} '
                f'XMLFile={xml_sixtesim} Background=yes RA={RA} Dec={Dec} ' 
                f'Exposure={exposure} clobber=yes')
        vprint(comm)
        output_sixtesim = run(comm, shell=True, capture_output=True)
        assert output_sixtesim.returncode == 0, f"sixtesim failed to run"
        assert os.path.exists(evtfile), f"sixtesim did not produce an output file"
        assert os.path.exists(photfile), f"sixtesim did not produce an output file"
        assert os.path.exists(impfile), f"sixtesim did not produce an output file"

## Do xifusim simulation  

### Get list of pixels with counts produced by sixtesim 

In [ ]:
#verbose=1
#read column PIXID from evtfile and save to a list of unique pixels
hdulist = fits.open(evtfile, mode='update')
evtdata = hdulist[1].data
pixels_with_impacts = np.unique(evtdata["PIXID"]) #photons coming from sources (PH_ID>-1) and background (PH_ID=-1)
vprint(f"Number of pixels with impacts: {len(pixels_with_impacts)}")
# all bkg impacts have same PH_ID identifier (-1)
# if more than one event with PH_ID=-1, change PH_ID of bkg impacts: if PH_ID==-1, change to consecutive negative number
if len(evtdata['PH_ID'][evtdata['PH_ID'] == -1]) > 1:
    phid_bkg = -1
    for i in range(len(evtdata)):
        if evtdata['PH_ID'][i] == -1:
            evtdata['PH_ID'][i] = phid_bkg
            phid_bkg -= 1
    #save changes to evtfile
hdulist.close()

# get pixels used and the events in each pixel
nimpacts_inpix = dict()
phid_impacts_inpix = dict()
for pixel in pixels_with_impacts:
    phid_impacts_inpix[pixel] = evtdata['PH_ID'][evtdata['PIXID'] == pixel]
    nimpacts_inpix[pixel] = len(phid_impacts_inpix[pixel])

#print number of impacts per pixel sorted by number of impacts
for key, value in sorted(nimpacts_inpix.items(), key=lambda item: item[1], reverse=True):
    vprint(f"Pixel {key}: {value} impacts")


#print the PH_ID of impacts in pixels
for key, value in phid_impacts_inpix.items():
    vprint(f"Pixel {key}: ")
    vprint(f"      PH_ID:{value}")

### Read sixtesim output data

In [ ]:
# open sixtesim ImpactList and read data 
hdulist = fits.open(impfile)
impdata = hdulist[1].data
hdulist.close()
# open sixtesim EvtList and read data (has been modified to include different PH_ID for background photons)
hdulist = fits.open(evtfile)
evtdata = hdulist[1].data
hdulist.close()

### For each pixel with impacts, do a xifusim simulation

In [ ]:
vprint(pixels_with_impacts)

#### Extract a piximpact file for each interesting pixel

In [ ]:
for ipixel in pixels_with_impacts:
    vprint(f"Checking existence of piximpact file for pixel {ipixel}")
    # create a subsample of the piximpact file selecting only those rows where PH_ID is in 
    # the list of the impacts in the pixel
    piximpactfile = f"{filestring}_pixel{ipixel}_piximpact.fits"
    
    # if file does not exist, create it
    if not os.path.exists(piximpactfile):
        # copy src impacts from impact file and bkgs from event file
        # background are not in the impact list (only in the event file)
        phid_impacts_inpix_srcs = phid_impacts_inpix[ipixel][phid_impacts_inpix[ipixel] > 0]
        
        # if there are no source impacts in the pixel, create a new table with only the background
        if len(phid_impacts_inpix_srcs) > 0:  # src impacts
            # create a mask to select only the rows with the impacts in this pixel
            mask = np.isin(impdata['PH_ID'], phid_impacts_inpix_srcs)
            # create a new table with the selected rows
            newtable = Table(impdata[mask])
        else: # create table to include only background impacts
            vprint(f"No source impacts in pixel {ipixel}")
            newtable = Table()
            # add columns TIME, SIGNAL, PH_ID and SRCID
            newtable['TIME'] = []
            newtable['ENERGY'] = []
            newtable['PH_ID'] = []
            newtable['SRC_ID'] = []
            
        # add the background impacts to the new table based on data in the event file
        # get indices of rows from event file where PH_ID < 0 (background)
        bkg_indices = np.where(evtdata['PH_ID'] < 0)[0]
        for ibkg in bkg_indices:
            if evtdata['PIXID'][ibkg] != ipixel:
                continue
            # create a new row in the new table
            newtable.add_row()
            # copy the bkg values from the event file (columns TIME, SIGNAL, PH_ID and SRCIF) 
            # to the new table (to columns TIME, ENERGY, PH_ID and SRCID)
            # Check first if newtable is empty (no source impacts)
            if len(newtable) == 0:
                newtable['TIME'] = evtdata['TIME'][ibkg]
                newtable['ENERGY'] = evtdata['SIGNAL'][ibkg]
                newtable['PH_ID'] = evtdata['PH_ID'][ibkg]
                newtable['SRC_ID'] = evtdata['SRC_ID'][ibkg]
            else:
                newtable['TIME'][-1] = evtdata['TIME'][ibkg]
                newtable['ENERGY'][-1] = evtdata['SIGNAL'][ibkg]
                newtable['PH_ID'][-1] = evtdata['PH_ID'][ibkg]
                newtable['SRC_ID'][-1] = evtdata['SRC_ID'][ibkg]
        # sort newtable according to TIME
        newtable.sort('TIME')
            
        # add new columns X,Y,U,V, GRADE1, GRADE2, TOTALEN with the value 0 and PIXID with the value of ipixel
        newtable['X'] = 0.
        newtable['Y'] = 0.
        newtable['U'] = 0.
        newtable['V'] = 0.
        newtable['GRADE1'] = 0
        newtable['GRADE2'] = 0
        newtable['TOTALEN'] = 0
        newtable['PIXID'] = ipixel

        # name the new table 'PIXELIMPACT'
        newtable.meta['EXTNAME'] = 'PIXELIMPACT'
    
        # write the new table to a new FITS file
        newtable.write(piximpactfile, format='fits', overwrite=True)

        # print the name of the new file rewriting the output line
        vprint(f"Created {piximpactfile} for pixel {ipixel}")


### Get XML for xifusim simulations

In [ ]:
# Find name of (unique) xml file in indir directory
xml_xifusim = glob.glob(f"./config*.xml")
if len(xml_xifusim) != 1:
    raise FileNotFoundError(f"Error: expected 1 XML file but found {len(xml_xifusim)}")
xml_xifusim = xml_xifusim[0]
vprint(f"Using XIFUSIM XML file: {xml_xifusim}")

### Run the xifusim simulation (with XML for single pixel)   
    - simulate time between min and max TIME in piximpact   
    - set PIXID to '1' for simulation   
    - xifusim simulation    
    - re-establish correct PIXID    
    - get the number of phsims in the pixel: check different values in PH_ID column

In [ ]:
#for each piximpact file, run xifusim
prebuffer = 1500
phsims_inpix = dict()
nphsims_inpix = dict()
skipped_photons_inpix = dict()
skipped_xifusim = []   

for ipixel in pixels_with_impacts:
    vprint(f"Selecting photons for pixel {ipixel}")
    vprint("====================================")
    # get total number of simulated photons in the pixel
    piximpactfile = f"{filestring}_pixel{ipixel}_piximpact.fits"
    with fits.open(piximpactfile) as hdulist_piximpact:
        piximpactdata = hdulist_piximpact[1].data.copy()
    phsims_inpix[ipixel] =  piximpactdata['PH_ID']
    nphsims_inpix[ipixel] = len(phsims_inpix[ipixel])

    # if no more than 1 impact in the pixel, skip the simulation
    if nimpacts_inpix[ipixel] <= 1:
        vprint(f"  Skipping simulation for pixel {ipixel} with {nimpacts_inpix[ipixel]} impact")
        skipped_xifusim.append(ipixel)
    else:  # check if there are close events in the pixel
        vprint(f"  Checking if needed simulation for pixel {ipixel} with {nimpacts_inpix[ipixel]} impact")
        PH_ID_toxifusim = []
        PH_ID_skipped = []
        time_diff = np.diff(piximpactdata['TIME'])
        close_photons = time_diff < 100 / sampling_rate

        for i in range(len(piximpactdata)):
            if ((i == 0 and close_photons[i]) or  
                (i == len(piximpactdata) - 1 and close_photons[i - 1]) or 
                (0 < i < len(piximpactdata) - 1 and (close_photons[i] or close_photons[i - 1]))):
                PH_ID_toxifusim.append(piximpactdata['PH_ID'][i])
                vprint(f"  Photon {piximpactdata['PH_ID'][i]} is close to another photon previous or next in time")
            else:
                PH_ID_skipped.append(piximpactdata['PH_ID'][i])

        skipped_photons_inpix[ipixel] = PH_ID_skipped
        #vprint(f"{len(PH_ID_toxifusim)} Photons for xifusim: {PH_ID_toxifusim}")
        #vprint(f"{len(PH_ID_skipped)} Skipped photons in pixel {ipixel}: {skipped_photons_inpix[ipixel]}")
        vprint(f"    {len(PH_ID_skipped)} Skipped photons in pixel {ipixel}")
        vprint(f"    {len(PH_ID_toxifusim)} Photons for xifusim")

        # if no photons closer than 100/sampling_rate in the pixel, skip the simulation 
        if len(PH_ID_toxifusim) == 0:
            vprint(f"    No photons closer than 100/sampling_rate in pixel {ipixel}: skipping simulation")
            skipped_xifusim.append(ipixel)
            continue
        # if there is no xifusim file for the pixel, create it
        xifusimfile = f"{filestring}_pixel{ipixel}_xifusim.fits"
        if not os.path.exists(xifusimfile):
            # create a new (reduced) FITS piximpact file keeping only those PH_ID ...
            # ... where TIME of photons is closer than 100/sampling_rate (samples)
            piximpactfile_toxifusim = f"{filestring}_pixel{ipixel}_piximpact_toxifusim.fits"
            if not os.path.exists(piximpactfile_toxifusim):
                # create a new table with the selected rows
                #mask = np.isin(piximpactdata['PH_ID'], PH_ID_toxifusim[PH_ID_toxifusim != 0])
                mask = np.isin(piximpactdata['PH_ID'], PH_ID_toxifusim)
                newtable = Table(piximpactdata[mask])
                # replace PIXID column with value '1' (xifusim requirement)
                newtable['PIXID'] = 1
                # name the new table 'PIXELIMPACT'
                newtable.meta['EXTNAME'] = 'PIXELIMPACT'
                # write the new table to a new FITS file
                newtable.write(piximpactfile_toxifusim, format='fits', overwrite=True)
                vprint(f"  Created {piximpactfile_toxifusim} for pixel {ipixel}")

            # use reduced piximpact file to run xifusim
            with fits.open(piximpactfile_toxifusim) as hdulist_toxifusim:
                piximpactdata_toxifusim = hdulist_toxifusim[1].data.copy()
            #calculate minimum and maximum time for impacts in the pixel
            mintime = np.min(piximpactdata_toxifusim['TIME'])
            maxtime = np.max(piximpactdata_toxifusim['TIME'])
            expos_init = mintime - 2.*prebuff_xifusim/sampling_rate
            expos_fin = maxtime + 0.1
            
            #create xifusim name based on input parameters    
            comm = (f'xifusim PixImpList={piximpactfile_toxifusim} Streamfile={xifusimfile} '
                    f'tstart={expos_init} tstop={expos_fin} '
                    f'trig_reclength=12700 '
                    f'trig_n_pre={prebuff_xifusim} '
                    f'trig_n_suppress=8192 '
                    f'trig_maxreclength=100000 '
                    f'XMLfilename={xml_xifusim} clobber=yes ')
            
            vprint(f"  Doing simulation for pixel {ipixel} with {len(piximpactdata_toxifusim)} impacts ({nimpacts_inpix[ipixel]} TOTAL impacts)")
            #vprint(f"Running {comm}")
            output_xifusim = run(comm, shell=True, capture_output=True)
            assert output_xifusim.returncode == 0, f"xifusim failed to run: {comm}"
            assert os.path.exists(xifusimfile), f"xifusim did not produce an output file"

            # re-write correct PIXID in the xifusim file
            with fits.open(xifusimfile, mode='update') as hdulist:
                xifusimdata = hdulist["TESRECORDS"].data
                xifusimdata['PIXID'] = ipixel
                hdulist.flush()

## DO reconstruction with SIRENA

### get LIBRARY adequate to XML file

In [ ]:
# Find name of (unique) library file in indir directory compatible with xifusim XML file
lib_sirena = glob.glob(f"./*library*")
if len(lib_sirena) != 1:
    raise FileNotFoundError(f"Expected 1 LIBRARY file but found {len(lib_sirena)}")
lib_sirena = lib_sirena[0]
vprint(f"Using LIBRARY file: {lib_sirena}")

### run `tesrecons`

In [ ]:
if recons:
    for ipixel in pixels_with_impacts:
        # if pixel was skipped in xifusim, skip reconstruction
        if ipixel in skipped_xifusim:
            continue
        xifusimfile = f"{filestring}_pixel{ipixel}_xifusim.fits"
        reconsfile = f"{filestring}_pixel{ipixel}_sirena.fits"
        if not os.path.exists(reconsfile):
            comm = (f"tesrecons Recordfile={xifusimfile} "
                f" TesEventFile={reconsfile}"
                f" LibraryFile={lib_sirena}"
                f" XMLFile={xml_xifusim}"
                f" clobber=yes"
                f" EnergyMethod=OPTFILT"
                f" OFStrategy=BYGRADE"
                f" filtEeV=6000"
                f" OFNoise=NSD"
            )
            vprint(f"Doing reconstruction for pixel {ipixel}", end='\r')
            #vprint(f"Running {comm}")
            output_tesrecons = run(comm, shell=True, capture_output=True)
            assert output_tesrecons.returncode == 0, f"tesrecons failed to run"
            assert os.path.exists(reconsfile), f"tesrecons did not produce an output file"
            

### check missing or misreconstructed photons

In [ ]:
verbose=1
if recons:
    # Check how many photons were reconstructed: compare the number of impacts in the pixel with the number of reconstructed photons
    ph_non_recons_inpix = dict()
    ph_bad_recons_inpix = dict()
    nextra_recons_inpix = dict()
    nbad_recons_inpix = dict()
    nnon_recons_inpix = dict()
    nrecons_inpix = dict()
    
    nrecons_total = 0
    nimpacts_total = 0
    nbad_recons_total = 0
    nnon_recons_total = 0
    
    for ipixel in pixels_with_impacts:
    #for ipixel in [776,]:
        # if pixel was skipped in xifusim (1 impact or separated impacts)
        if ipixel in skipped_xifusim:
            # get the number of reconstructed photons
            nrecons_inpix[ipixel] = nphsims_inpix[ipixel] 
            nextra_recons_inpix[ipixel] = 0  
            ph_non_recons_inpix[ipixel] = np.array([])
            ph_bad_recons_inpix[ipixel] = np.array([])
            nnon_recons_inpix[ipixel] = 0
            nbad_recons_inpix[ipixel] = 0
        else:
            #read SIRENA file
            reconsfile = f"{filestring}_pixel{ipixel}_sirena.fits"
            hdulist = fits.open(reconsfile)
            reconsdata = hdulist[1].data
            PH_ID = reconsdata['PH_ID']
            hdulist.close()
            # get the number of reconstructed photons
            nrecons_inpix[ipixel] = len(reconsdata) + len(skipped_photons_inpix[ipixel])
            nextra_recons_inpix[ipixel] = 0

            # read PIXIMPACT file (for TIME column)
            hdulist = fits.open(f"{filestring}_pixel{ipixel}_piximpact.fits")
            piximpactdata = hdulist[1].data
            hdulist.close()

            # inititalize the lists of photons 
            ph_non_recons_inpix[ipixel] = phsims_inpix.get(ipixel, [])
            ph_bad_recons_inpix[ipixel] = np.array([])
            nnon_recons_inpix[ipixel] = 0
            nbad_recons_inpix[ipixel] = 0

            
            vprint(f"Pixel {ipixel}: ")
            vprint(f"      {nimpacts_inpix[ipixel]} impacts")
            vprint(f"      {nphsims_inpix[ipixel]} simulated photons")
            vprint(f"      {nrecons_inpix[ipixel]} (inititally)reconstructed photons")
            
            # save sequences of xifusim colum PH_ID with more than 1 non zero elements
            xifusimfile = f"{filestring}_pixel{ipixel}_xifusim.fits"
            # read PH_ID column
            hdulist = fits.open(xifusimfile)
            xifusimdata = hdulist["TESRECORDS"].data
            PH_ID_xifusim = xifusimdata['PH_ID']
            hdulist.close()

            irows_checked = []
            for irow in range(len(PH_ID)):
                if irow in irows_checked:
                    vprint(f"    SIRENA: row {irow+1} in pixel {ipixel} already checked")
                    continue
                
                vprint(f"SIRENA: Checking row {irow+1} in pixel {ipixel}")
                # get number of values in the array PH_ID[irow] that are /= 0
                nphotons = np.count_nonzero(PH_ID[irow])
                
                vprint(f"    SIRENA: PH_ID[row={irow+1}]: {PH_ID[irow]}, nphotons: {nphotons}")
                
                if nphotons == 1:
                    # remove PH_ID[irow] value from the list of non-reconstructed photons
                    vprint(f"    SIRENA: Removing photon {PH_ID[irow][0]} from the list of non-reconstructed photons")
                    ph_non_recons_inpix[ipixel] = np.delete(ph_non_recons_inpix[ipixel], np.where(ph_non_recons_inpix[ipixel] == PH_ID[irow][0]))
                else:
                    # if nphotons==2, the sequence of photons in xifusim == sirena
                    ph_full_sequence_xifusim = PH_ID[irow][np.nonzero(PH_ID[irow])]
                    # get all the indices of the rows with the same values as PH_ID[irow]
                    indices_same_photons = np.where((PH_ID == PH_ID[irow]).all(axis=1))[0]
                    vprint(f"    SIRENA: rows with same photons: {indices_same_photons+1}")
                    # add indices_same_photons values to the list of checked rows
                    irows_checked.extend(indices_same_photons)
                    
                    if len(indices_same_photons) < nphotons:
                        vprint(f"    SIRENA: Warning: missed {nphotons-len(indices_same_photons)} photons in the list {PH_ID[irow]}")
                        # try to identify missed photon(s) looking at the TIME column
                        xifusim_sirena_photons = dict()
                        for ph in ph_full_sequence_xifusim:
                            # look for the photon in the piximpact file and get TIME value
                            idx_ph = np.where(piximpactdata['PH_ID'] == ph)[0]
                            time_ph_piximpact = piximpactdata['TIME'][idx_ph]
                            # look for the photon in the SIRENA file and get TIME value: compare with TIME in piximpact file
                            for idx in indices_same_photons:
                                time_ph_sirena = reconsdata['TIME'][idx]
                                # if close than 20 samples, consider it as the same photon
                                if abs(time_ph_piximpact-time_ph_sirena)*sampling_rate < 20:
                                    xifusim_sirena_photons[ph] = idx
                                    break
                        vprint(f"    SIRENA: xifusim_sirena_photons: {xifusim_sirena_photons}")
                        
                        # look if the xifusim_sirena_photons dictionary has duplicated values: identify keys with the same value
                        sirena_phs = list(xifusim_sirena_photons.values())
                        xifusim_phs = list(xifusim_sirena_photons.keys())
                        unique_sirena_phs = list(set(sirena_phs))
                        # identify the xifusim photons with same sirena identification
                        # loop over the indices of unique sirena_phs:
                        for uni_ph in unique_sirena_phs:
                            i = sirena_phs.index(uni_ph)
                            #vprint(f"i={i}, sirena_ph={sirena_phs[i]}, uni_ph={uni_ph}")
                            if sirena_phs.count(sirena_phs[i]) > 1:
                                # get xifusim photons with the same sirena identification
                                indices_mixed_photons = [j for j, x in enumerate(sirena_phs) if x == sirena_phs[i]]  
                                vprint(f"    SIRENA: Warning - indices_mixed_photons: {indices_mixed_photons}")
                                vprint(f"    SIRENA: Warning XIFUSIM photons with the same SIRENA row: {[xifusim_phs[j] for j in indices_mixed_photons]}")
                                # remove the first multiple photon from the list of non-reconstructed photons
                                photon_to_remove = xifusim_phs[indices_mixed_photons[0]]
                                # and add the photon to the list of compromised photons
                                vprint(f"    SIRENA: adding photon {photon_to_remove} to the list of compromised photons")
                                ph_bad_recons_inpix[ipixel] = np.append(ph_bad_recons_inpix[ipixel], photon_to_remove)
                                # leave other mixed photons in the list of non-reconstructed photons
                                for j in range(1,len(indices_mixed_photons)):
                                    vprint(f"    SIRENA: leaving photon {xifusim_phs[indices_mixed_photons[j]]} in the list of non-reconstructed photons")
                            else:
                                indices_mixed_photons = []
                                photon_to_remove = xifusim_phs[i]
                            # remove the photons found in the SIRENA file from the list of non-reconstructed photons
                            vprint(f"    SIRENA: Removing photon {photon_to_remove} from the list of non-reconstructed photons")
                            ph_non_recons_inpix[ipixel] = np.delete(ph_non_recons_inpix[ipixel], np.where(ph_non_recons_inpix[ipixel] == photon_to_remove))
                    elif len(indices_same_photons) > nphotons:
                        vprint(f"    SIRENA: Warning: more detections ({len(indices_same_photons)}) than photons ({nphotons}) in the list {ph_full_sequence_xifusim}") 
                        nextra_recons_inpix[ipixel] += (len(indices_same_photons) - nphotons)
                    else:
                        # remove ph_full_sequence_xifusim values from the list of non-reconstructed photons
                        for ph in ph_full_sequence_xifusim:
                            vprint(f"    SIRENA: Removing photon {ph} from the list of non-reconstructed photons")
                            ph_non_recons_inpix[ipixel] = np.delete(ph_non_recons_inpix[ipixel], np.where(ph_non_recons_inpix[ipixel] == ph))
                    # end comparison of nphotons in xifusim and nphotons in sirena
                # end sirena row with more than 1 photon
                # remove skipped photons in this pixel from the list of non-reconstructed photons
                ph_non_recons_inpix[ipixel] = np.setdiff1d(ph_non_recons_inpix[ipixel], skipped_photons_inpix[ipixel])
                
            # end loop over rows in SIRENA file
            nnon_recons_inpix[ipixel] = len(ph_non_recons_inpix[ipixel])
            nbad_recons_inpix[ipixel] = len(ph_bad_recons_inpix[ipixel])
        # end if pixel was skipped in xifusim (sirena file exists or not)
        nbad_recons_total += nbad_recons_inpix[ipixel]
        nrecons_total += nrecons_inpix[ipixel] - nextra_recons_inpix[ipixel] 
        nnon_recons_total += nnon_recons_inpix[ipixel]       
        nimpacts_total += nimpacts_inpix[ipixel]
    
        if verbose > 0:
            print(f"Summary for Pixel {ipixel}: ")
            print(f"=====================================")
            print(f"      {nimpacts_inpix[ipixel]} impacts")
            print(f"      {nphsims_inpix[ipixel]} simulated photons")
            print(f"      {nextra_recons_inpix[ipixel]} (extra)reconstructed photons")
            print(f"      {nbad_recons_inpix[ipixel]} compromised-recons photons: {ph_bad_recons_inpix[ipixel]}")
            print(f"      {nrecons_inpix[ipixel]} (final)reconstructed photons")
            print(f"      {nnon_recons_inpix[ipixel]} missed photons: {ph_non_recons_inpix[ipixel]}")
            print(f"      {nrecons_total} Accumulated reconstructed photons")

            # print missing photons in the pixel
            if nrecons_inpix[ipixel] < nphsims_inpix[ipixel]:
                # identify in which row of PH_ID_xifusim the missing photons are
                hdulist = fits.open(f"{filestring}_pixel{ipixel}_xifusim.fits")
                xifusimdata = hdulist["TESRECORDS"].data
                PH_ID_xifusim = xifusimdata['PH_ID']
                hdulist.close()
                idx_phs = []
                for ph in ph_non_recons_inpix[ipixel]:
                    idx_ph = np.where(PH_ID_xifusim == ph)[0]
                    idx_phs.append(idx_ph)
                    print(f"      Missed photons: {ph} in xifusim rows {np.array(idx_ph)+1}")

    # end loop over pixels


    # calculate fraction of lost photons: one is lost and the other one is piledup
    #fraction_lost = 1. - nrecons_total/nimpacts_total 
    fraction_lost = nnon_recons_total/nimpacts_total
    fraction_badrecons = nbad_recons_total/nimpacts_total
    

In [ ]:
if recons:
    #save info to a file (create if it does not exist): 
    # simulation, flux_mcrab, exposure, filter, focus, number of pixels, number of impacts, number of reconstructed photons, fraction_lost
    infofile = f"info_{filter}_{focus}_test.csv"
    vprint(f"Saving information to {infofile}")
    vprint(f"Simulation {sim_number}:")
    vprint(f"      Flux: {flux_mcrab:.3f} mCrab")
    vprint(f"      Exposure: {exposure:.2f} s")
    vprint(f"      Filter: {filter}")
    vprint(f"      Focus: {focus}")
    vprint(f"      Number of pixels: {len(pixels_with_impacts)}")
    vprint(f"      Number of impacts: {nimpacts_total}")
    vprint(f"      Number of reconstructed photons: {nrecons_total}")
    vprint(f"      Number of lost photons: {nnon_recons_total}")
    vprint(f"      Number of bad reconstructed photons: {nbad_recons_total}")
    vprint(f"      Fraction of lost photons: {fraction_lost:.2e}")
    vprint(f"      Fraction of bad reconstructed photons: {fraction_badrecons:.2e}")


## Save results   

In [ ]:
# Detailed information about missing photons for the simulation (for each pixel)
if recons:
    # save info to pandas table
    # First column: pixel ID (integer)
    # Second column: list of non-reconstructed photons (integer)
    # Third column: list of bad-reconstructed photons (integer)
    info_table = pd.DataFrame(columns=['Pixel', 'Non-reconstructed photons', 'Bad-reconstructed photons'], dtype=object)
    for ipixel in pixels_with_impacts:
        if len(ph_non_recons_inpix[ipixel]) > 0 or len(ph_bad_recons_inpix[ipixel]) > 0:
            # add row to the table
            info_table.loc[len(info_table)] = [ipixel, list(ph_non_recons_inpix[ipixel].astype(int)), list(ph_bad_recons_inpix[ipixel].astype(int))]
    vprint(info_table)
    # save table to a csv file
    infofile = f"{outDir}/00_info_{filter}_{focus}_sim{sim_number}_missing.csv"
    info_table.to_csv(infofile, index=False)
    vprint(f"Information saved to {infofile}")


In [ ]:
#global info
infofile = f"info_{filter}_{focus}_global.csv"
if recons:
    if not os.path.exists(infofile):
        with open(infofile, 'w') as f:
            f.write(f"simulation,flux[mcrab],exposure[s],filter,focus,Npixels,Nimpacts,Nrecons,Missing,Nbadrecons,fraction_lost[%],fraction_badreconstructed[%]\n")
    with open(infofile, 'a') as f:
        f.write(f"{sim_number},{flux_mcrab:.3f},{exposure},{filter},{focus},{len(pixels_with_impacts)},"
                f"{nimpacts_total},{nrecons_total},{nnon_recons_total},{nbad_recons_total},"
                f"{100*fraction_lost:.2e},{100*fraction_badrecons:.2e}\n")